In [2]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [38]:
app = Flask(__name__)
CORS(app,resources={r"/*": {"origins": ["http://127.0.0.1:5000"]}})
with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)

engine = create_engine("sqlite:///Resources/votes.sqlite")
base = automap_base()
base.prepare(autoload_with=engine)
# inspector = inspect(engine)
# print(inspector.get_table_names())
types = base.classes.Favorite_Cuisines
# print(f"Reflected table name: {types.__table__.name}")
# print(f"Columns: {types.__table__.columns}")
Session = sessionmaker(bind=engine)
session = Session()
# session.add(types(restaurant_type = "mexican", votes_cast = 0))
# session.add(types(restaurant_type = "italian", votes_cast = 0))
# session.add(types(restaurant_type = "thai", votes_cast = 0))
# session.commit()
testq = session.query(types).all()
print([i.votes_cast for i in testq if True])
session.close()

def pull_vote_totals():
    session = Session()
    boop = session.query(types).all()
    democracy = {"mexican":0, "italian": 0, "thai":0}
    
    for i in boop:
        print(i.restaurant_type)
        democracy[i.restaurant_type] = i.votes_cast   
        print(i.votes_cast) 
    session.close()
    print(democracy)
    return democracy
    

def add_vote_to_db(vote,types):
    session = Session()
    tp = session.query(types).filter(types.restaurant_type == vote).first()
    # for t in tp:
        # t.votes_cast += 1
    tp.votes_cast += 1
    session.commit()
    session.close()
    return json.dumps("vote added to db")
    

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    add_vote_to_db(vote,types)
    return vote
    
@app.route("/voteTot")
def jose():
    voteTots = pull_vote_totals()
    return json.dumps(voteTots)

@app.route("/data")
def josephine():
    return json.dumps(data)


if(__name__=="__main__"):
    app.run()

[3, 0, 0]
closing session
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


adding votes to db
the vote is: mexican
the len of tp is: 1
looping through tp
3
4
this is the result: [<sqlalchemy.ext.automap.Favorite_Cuisines object at 0x000002609062BF48>]


127.0.0.1 - - [13/Apr/2023 16:25:24] "GET / HTTP/1.1" 200 -


added mexican to db


...

pulling vote totals
mexican
10
italian
5
thai
7
{'mexican': 10, 'italian': 5, 'thai': 7}
pulled vote totals


127.0.0.1 - - [13/Apr/2023 16:27:10] "GET /voteTot HTTP/1.1" 200 -


pulling vote totalsthe vote is: italian 
the type is: <class 'sqlalchemy.ext.automap.Favorite_Cuisines'>
adding votes to db
the vote is: italian
mexican
10
italian
5
thai
7
{'mexican': 10, 'italian': 5, 'thai': 7}
pulled vote totals
the len of tp is: 1
looping through tp
5
6
this is the result: [<sqlalchemy.ext.automap.Favorite_Cuisines object at 0x0000026090C1A888>]


127.0.0.1 - - [13/Apr/2023 16:27:10] "GET /voteitalian HTTP/1.1" 200 -


added italian to db
